In [1]:
import pandas as pd
import sqlalchemy as sq
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option("display.max_columns",200)
from matplotlib import cm as cm
from sklearn import linear_model

from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

/Users/kylemix/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Helper Functions

This function takes a dataframe and a string, and returns a new dataframe with all columns with headers that begin with that string

In [2]:
def get_columns(df,key):
    headers_all = list(df.columns)
    headers = [name for name in headers_all if name[0:len(key)]==key]
    return df.loc[:,headers]

This function takes a question key table + a question key, and return the question associated with the key

In [3]:
def get_question(df,key):
    return df.loc[df.iloc[:,0]==key,:]

In [4]:
def get_question_text(key):
    q = df_key0.loc[df_key0.iloc[:,0]==str(key.strip()),'text']
    if len(q.values)>0:
        return q.values[0]
    else:
        return 'shitsfucked'


# Connect to database

In [5]:
engine = sq.create_engine('sqlite:///OKC_Processed_DB.db') #create connection to the database

# Get tables

In [6]:
df_num = pd.read_sql_table('numerical_attributes',engine)
df_rsp = pd.read_sql_table('question_responses',engine)
df_att = pd.read_sql_table('attributes_preferences',engine)
df_tst = pd.read_sql_table('test_responses',engine)
df_key0 = pd.read_sql_table('questions_key',engine)
df_key = df_key0[df_key0.loc[:,'N']>=31548]

In [7]:
tableList = [df_num,df_rsp,df_att,df_tst,df_key]
shapeList = []
for I in tableList:
    shapeList.append(I.shape)
print(shapeList)



[(68372, 56), (68371, 578), (68371, 146), (68371, 47), (231, 10)]


### table shapes:

+ df_num , 'numerical_attributes' , (68371, 55)

+ df_rsp, question_responses, (68371, 577)

+ df_att, attributes_preferences (68371, 84)

+ df_tst, test_responses, (68371, 46)

+ df_key, questions_key, (231, 10)

# Modelling

In [8]:
str(get_question(df_key,'q19874').text.values[0])

'How long do your romantic relationships usually last?'

In [9]:
get_columns(df_rsp,'q34113').sum()

q34113_It's_okay,_if_it_is_not_abused    18029
q34113_Never_-_Get_a_job                 37271
q34113_No_problem                         9395
q34113_Okay_for_short_amounts_of_time     3676
dtype: float64

In [10]:
c1 = get_columns(df_rsp,'q19874')

In [11]:
def search_questions(df,key):
    indexlist = []
    key = str.upper(key)
    df.fillna(value=0,inplace=True)
    for i in range(len(df_key.iloc[:,1])):
        if key in str.upper(str(df_key.iloc[i,1])):
            indexlist.append(i)
        elif key in str.upper(str(df_key.iloc[i,2])):
            indexlist.append(i)
        elif key in str.upper(str(df_key.iloc[i,3])):
            indexlist.append(i)
        elif key in str.upper(str(df_key.iloc[i,4])):
            indexlist.append(i)
        elif key in str.upper(str(df_key.iloc[i,5])):
            indexlist.append(i)
        elif key in str.upper(str(df_key.iloc[i,6])):
            indexlist.append(i)
        elif key in str.upper(str(df_key.iloc[i,7])):
            indexlist.append(i)
        elif key in str.upper(str(df_key.iloc[i,8])):
            indexlist.append(i)
        elif key in str.upper(str(df_key.iloc[i,9])):
            indexlist.append(i)
    return df_key.iloc[indexlist,:]
#Example
search_questions(df_key,'sex')

/Users/kylemix/anaconda/lib/python3.5/site-packages/pandas/core/frame.py:2705: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,Unnamed: 0,text,option_1,option_2,option_3,option_4,N,Type,Order,Keywords
3,d_orientation,Sexual orientation,0,0,0,0,66365,0,0,0
29,q325,Would you consider having an open relationship...,Yes,No,0,0,54443,O,0,sex/intimacy; preference
36,q1597,Would you consider sleeping with someone on th...,Yes,No,0,0,52509,O,0,sex/intimacy; preference
42,q24375,Say you've started seeing someone you really l...,1-2 dates,3-5 dates,6 or more dates,Only after the wedding,51072,O,0,sex/intimacy; religion/superstition; preference
46,q35,"Regardless of future plans, what's more intere...",Sex,Love,0,0,50384,N,0,sex/intimacy
47,q12964,Would you need to sleep with someone before yo...,Yes,No,0,0,50115,O,0,sex/intimacy; preference
53,q149,Which makes for a better relationship?,Passion,Dedication,0,0,49355,N,0,sex/intimacy
54,q70,Do you think homosexuality is a sin?,Yes,No,0,0,48983,O,0,religion/superstition
69,q12605,"Once you're intimate, how often would you and ...",Every day,About every other day,Once or twice a week,A few times a month or less,46098,O,0,sex/intimacy
76,q20135,How do you feel about falling in love?,I love it and want it very much,I try to avoid it,I like to just let it happen,I'm indifferent / not sure,44683,O,"2, (3, 4), 1",sex/intimacy; preference


In [12]:
dfSex = search_questions(df_key,'virgin')
dfPartners = search_questions(df_key,'open')


/Users/kylemix/anaconda/lib/python3.5/site-packages/pandas/core/frame.py:2705: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [13]:
sql1 = "SELECT AVG(num.CA) FROM numerical_attributes num INNER JOIN attributes_preferences att ON num.userID = att.userID WHERE att.d_education_type_Two_year_college = 1"  

sql2 = "SELECT rsp FROM attributes_preferences att INNER JOIN question_responses rsp ON att.userID = rsp.userID WHERE att.d_Black = 1"

In [14]:
lawyers = "SELECT * FROM question_responses rsp WHERE rsp.userID IN (SELECT userID FROM attributes_preferences WHERE d_education_type_Law_school = 1)"
fatties = "SELECT * FROM question_responses rsp WHERE rsp.userID IN (SELECT userID FROM attributes_preferences WHERE d_bodytype_Overweight = 1)"
taken = "SELECT * FROM question_responses rsp WHERE rsp.userID IN (SELECT userID FROM attributes_preferences WHERE d_relationship_Seeing_Someone = 1)"
sales =  "SELECT * FROM question_responses rsp WHERE rsp.userID IN (SELECT userID FROM attributes_preferences WHERE d_job_Sales_\/_Marketing = 1)"
wellFareHaters = "SELECT * FROM question_responses rsp WHERE rsp.q34113_Never_-_Get_a_job = 1"

In [15]:
# result = engine.execute("SELECT * FROM question_responses LIMIT 1 ")
# dfx = pd.read_sql_query("SELECT * FROM questions_key WHERE Keywords LIKE '%preference%' ",engine)
# well = pd.read_sql_query(wellFareHaters,engine)


In [16]:
# lawyerCorrelations = dfx.corr()

In [19]:
def nullAutoCorr(x):
    if x == 1.0:
        return np.nan
    else:
        return x

In [20]:
def split_questionID(x):
    bb = x.split('_')
    return bb[0]

def splt_Q_Response(x):
    bb = x.split('_')
    ret = bb[1:]
    return ' '.join(ret)

def mostNegTable(df,newdf):
    newdf.columns=['testQuestionResponseNeg','QuesIDTestNeg','testResponseTextNeg','corrNeg']
    for x in range(1,df.shape[1]-1):
        tt = df.iloc[:,x].nsmallest(2)
        a = tt.reset_index()
        qResp = a.iloc[1,0]
        respCorr = a.iloc[1,1]
        qID = split_questionID(qResp)
        respText = splt_Q_Response(qResp)
        newdf.loc[x]= str(qResp), str(qID), str(respText), str(respCorr)  

        
def mpt(df,newdf):
    newdf.columns=['testQuestionResponsePos','QuesIDTestPos','testResponseTextPos','corrPos']
    for x in range(1,df.shape[1]-1):
        tt = df.iloc[:,x].nlargest(2)
        a = tt.reset_index()
        qResp = a.iloc[0,0]
        respCorr = a.iloc[0,1]
        qID = split_questionID(qResp)
        respText = splt_Q_Response(qResp)
        newdf.loc[x]= str(qResp), str(qID), str(respText), str(respCorr) 
        

In [21]:
# list of pridictor questions

df_q18698 = get_columns(df_tst,'q18698')
df_q1062 = get_columns(df_rsp,'q1062')
df_q1128 = get_columns(df_rsp,'q1128')
df_q123 = get_columns(df_rsp,'q123')
df_q123['blank1'] = np.zeros((68371,1))
df_q123['blank2'] = np.zeros((68371,1))
df_q136 = get_columns(df_rsp,'q136')
df_q136['blank3'] = np.zeros((68371,1))
df_q136['blank4'] = np.zeros((68371,1))
df_q15280 = get_columns(df_rsp,'q15280')
df_q15280['blank5'] = np.zeros((68371,1))
df_q15280['blank6'] = np.zeros((68371,1))
df_q1597 = get_columns(df_rsp,'q1597')
df_q1597['blank7'] = np.zeros((68371,1))
df_q1597['blank8'] = np.zeros((68371,1))
df_q19874 = get_columns(df_rsp,'q19874')
df_q274 = get_columns(df_rsp,'q274_')
df_q35 = get_columns(df_rsp,'q35_')
df_q35['blank9'] = np.zeros((68371,1))
df_q35['blank10'] = np.zeros((68371,1))
df_q358084 = get_columns(df_rsp,'q358084')
df_q358084['blank11'] = np.zeros((68371,1))
df_q358084['blank12'] = np.zeros((68371,1))
df_q37693 = get_columns(df_rsp,'q37693')
df_q393 = get_columns(df_rsp,'q393_')
df_q393['blank13'] = np.zeros((68371,1))
df_q393['blank14'] = np.zeros((68371,1))
df_q4018 = get_columns(df_rsp,'q4018')
df_q4018['blank15'] = np.zeros((68371,1))
df_q4018['blank16'] = np.zeros((68371,1))
df_q43261 = get_columns(df_rsp,'q43261')
df_q60100 = get_columns(df_rsp,'q60100')
df_q60100['blank17'] = np.zeros((68371,1))
df_q60100['blank18'] = np.zeros((68371,1))
df_q66506 = get_columns(df_rsp,'q66506')
df_q66506['blank19'] = np.zeros((68371,1))
df_q77 = get_columns(df_rsp,'q77')
df_q79 = get_columns(df_rsp,'q79')
df_q81 = get_columns(df_rsp,'q81_')
dfPred = pd.concat([df_q81,df_q79,df_q77,df_q66506,df_q60100,
                    df_q43261,df_q4018,df_q393,df_q37693,df_q358084,df_q35,df_q274,df_q19874,df_q1597,
                   df_q15280,df_q136,df_q123,df_q1128,df_q1062, df_q18698],axis=1)

In [22]:
dfPred.shape

(68371, 80)

In [23]:
dfPred.shape


# 41, 42 

(68371, 80)

In [24]:
dftest.head()

NameError: name 'dftest' is not defined

In [25]:
dfPred.columns

Index(['q81_Lots', 'q81_Never', 'q81_Rarely', 'q81_Sometimes',
       'q79_I_smoke_occasionally.', 'q79_I_smoke_regularly.',
       'q79_I_smoked_in_the_past,_but_no_longer.', 'q79_Never.', 'q77_Never',
       'q77_Rarely', 'q77_Sometimes', 'q77_Very_often',
       'q66506_Not_important.', 'q66506_Somewhat_important.',
       'q66506_Very_important.', 'blank19', 'q60100_No', 'q60100_Yes',
       'blank17', 'blank18',
       'q43261_I_almost_always_do_what_my_parents_think_is_best.',
       'q43261_I_always_do_what_my_parents_say.', 'q43261_I_am_my_own_person.',
       'q43261_I_consider_their_opinion_but_go_my_own_way.', 'q4018_No',
       'q4018_Yes', 'blank15', 'blank16', 'q393_No', 'q393_Yes', 'blank13',
       'blank14', 'q37693_No.', 'q37693_Yes,_I'd_be_playing_with_them.',
       'q37693_Yes,_but_I_don't_like_video_games.',
       'q37693_Yes,_but_I_would_not_play_that_much.', 'q358084_No',
       'q358084_Yes', 'blank11', 'blank12', 'q35_Love', 'q35_Sex', 'blank9',
       'blank

In [ ]:
get_columns(df_rsp,'q170849')
# d_gender_Woman

In [26]:
# what is your stance on open relationships?
y1 = df_rsp[['q325_Yes','q325_No']]
# what is your gender?
y2 = df_att[['d_gender_Man','d_gender_Woman']]
# do you google someone on the first date?
y3 = df_rsp[["q170849_No._Why_spoil_the_mystery?","q170849_Yes._Knowledge_is_power!"]]
# what is your stance on drug use?
y4 = get_columns(df_rsp,'q80_')
# orgasams are the most important part of sex y/n
y5 = get_columns(df_rsp,'q324')
# whach best describes you carfree or intense?
y6 = get_columns(df_rsp,'q49')
# is art important to you?
y7 = get_columns(df_rsp, 'q1766')
# which would you rather have?  flight or invisibility?
y8 = get_columns(df_rsp, 'q445141')
# are you annoyed by super logical people?
y9 = get_columns(df_rsp, 'q301')
# which do you spend more money on clothes or food?
y10 = get_columns(df_rsp, 'q74')
# would you date someone who kept a gun in the house?
y11 = get_columns(df_rsp, 'q31055')
# would not enough space or not enough affection bother you more in a relationship?
y12 = get_columns(df_rsp, 'q491')
# were you picked on a lot in school?
y13 = get_columns(df_rsp, 'q313640')

In [27]:

dfRon = pd.concat([y1,dfPred], axis=1)

dfBob = pd.concat([y4,dfPred],axis=1)

dfDan = pd.concat([y3,dfPred],axis=1)

dfKyle = pd.concat([y2,dfPred],axis=1)

dfOrg = pd.concat([y5,dfPred], axis=1)

dfInt = pd.concat([y6,dfPred], axis=1)

dfArt = pd.concat([y7,dfPred], axis=1)

dfFlight = pd.concat([y8,dfPred], axis=1)

dfLogic = pd.concat([y9,dfPred], axis=1)

dfClothes = pd.concat([y10,dfPred], axis=1)

dfGuns = pd.concat([y11,dfPred], axis=1)

dfSpace = pd.concat([y12,dfPred], axis=1)

dfBullied = pd.concat([y13,dfPred], axis=1)

In [28]:

dfRon[(dfRon['q325_Yes']==0)&(dfRon['q325_No']==0)]=np.nan
dfKyle[(dfKyle['d_gender_Man']==0)&(dfKyle['d_gender_Woman']==0)]=np.nan
dfDan[(dfDan["q170849_No._Why_spoil_the_mystery?"]==0)&(dfDan["q170849_Yes._Knowledge_is_power!"]==0)]=np.nan
dfBob[(dfBob.iloc[:,0]==0)&(dfBob.iloc[:,1]==0)&(dfBob.iloc[:,2]==0)&(dfBob.iloc[:,3]==0)]=np.nan
dfOrg[(dfOrg.iloc[:,0]==0)&(dfOrg.iloc[:,1]==0)]=np.nan
dfInt[(dfInt.iloc[:,0]==0)&(dfInt.iloc[:,1]==0)]=np.nan
dfArt[(dfArt.iloc[:,0]==0)&(dfArt.iloc[:,1]==0)]=np.nan
dfFlight[(dfFlight.iloc[:,0]==0)&(dfFlight.iloc[:,1]==0)]=np.nan
dfLogic[(dfLogic.iloc[:,0]==0)&(dfLogic.iloc[:,1]==0)]=np.nan
dfClothes[(dfClothes.iloc[:,0]==0)&(dfClothes.iloc[:,1]==0)]=np.nan
dfGuns[(dfGuns.iloc[:,0]==0)&(dfGuns.iloc[:,1]==0)]=np.nan
dfSpace[(dfSpace.iloc[:,0]==0)&(dfSpace.iloc[:,1]==0)]=np.nan
dfBullied[(dfBullied.iloc[:,0]==0)&(dfBullied.iloc[:,1]==0)]=np.nan

In [29]:
dfRon = dfRon.dropna()
dfKyle = dfKyle.dropna()
dfDan = dfDan.dropna()
dfBob = dfBob.dropna()
dfOrg = dfOrg.dropna()
dfInt = dfInt.dropna()
dfArt = dfArt.dropna()
dfFlight = dfFlight.dropna()
dfLogic = dfLogic.dropna()
dfClothes = dfClothes.dropna()
dfGuns = dfGuns.dropna()
dfSpace = dfSpace.dropna()
dfBullied = dfBullied.dropna()

In [30]:
ron_y = dfRon.iloc[:,0:1]
ron_x = dfRon.iloc[:,2:]
kyle_y = dfKyle.iloc[:,0:1]
kyle_x = dfKyle.iloc[:,2:]
dan_y = dfDan.iloc[:,0:1]
dan_x = dfDan.iloc[:,2:]
bob_y = dfBob.iloc[:,3:4]
bob_x = dfBob.iloc[:,4:]
org_y = dfOrg.iloc[:,0:1]
org_x = dfOrg.iloc[:,2:]
int_y = dfInt.iloc[:,0:1]
int_x =dfInt.iloc[:,2:]
art_y = dfArt.iloc[:,0:1]
art_x =dfArt.iloc[:,2:]
flight_y = dfFlight.iloc[:,0:1]
flight_x =dfFlight.iloc[:,2:]
logic_y = dfLogic.iloc[:,0:1]
logic_x =dfLogic.iloc[:,2:]
clothes_y = dfClothes.iloc[:,0:1]
clothes_x = dfClothes.iloc[:,2:]
guns_y = dfGuns.iloc[:,0:1]
guns_x = dfGuns.iloc[:,2:]
space_y = dfSpace.iloc[:,0:1]
space_x = dfSpace.iloc[:,2:]
bullied_y = dfBullied.iloc[:,0:1]
bullied_x = dfBullied.iloc[:,2:]

In [50]:
bob_y.head()

,q80_I_never_do_drugs.
0,1
5,1
8,1
10,1
12,1


In [32]:
# how do you feel about open relationships?
X_trainR, X_testR, y_trainR, y_testR = train_test_split(ron_x, ron_y, test_size=.3,random_state=0)
modR = linear_model.LogisticRegression()
fitR = modR.fit(X_trainR, y_trainR)
y_predR = fitR.predict_proba(X_testR)
roc_auc_score(y_testR,y_predR[:,1])

/Users/kylemix/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.84875446544920385

In [33]:
# do you google someone before the first date?
X_trainD, X_testD, y_trainD, y_testD = train_test_split(dan_x, dan_y, test_size=.3,random_state=0)
modD = linear_model.LogisticRegression()
fitD = modD.fit(X_trainD, y_trainD)
y_predD = fitD.predict_proba(X_testD)
roc_auc_score(y_testD,y_predD[:,1])

/Users/kylemix/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.60960339224287607

In [34]:
# male or female?
X_trainK, X_testK, y_trainK, y_testK = train_test_split(kyle_x, kyle_y, test_size=.3,random_state=0)
modK = linear_model.LogisticRegression()
fitK = modK.fit(X_trainK, y_trainK)
y_predK = fitK.predict_proba(X_testK)
roc_auc_score(y_testK,y_predK[:,1])

/Users/kylemix/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.77391653191956677

In [35]:
# what is your stance on drug use?
X_trainB, X_testB, y_trainB, y_testB = train_test_split(bob_x, bob_y, test_size=.3,random_state=0)
modB = linear_model.LogisticRegression()
fitB = modB.fit(X_trainB, y_trainB)
y_predB = fitB.predict_proba(X_testB)
roc_auc_score(y_testB,y_predB[:,1])

/Users/kylemix/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.84401671153177926

In [51]:
fitB.classes_

array([ 0.,  1.])

In [36]:
# orgasms are the most important part of sex
X_traino, X_testo, y_traino, y_testo = train_test_split(org_x, org_y, test_size=.3,random_state=0)
modo = linear_model.LogisticRegression()
fito = modo.fit(X_traino, y_traino)
y_predo = fito.predict_proba(X_testo)
roc_auc_score(y_testo,y_predo[:,1])

/Users/kylemix/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.66415554899781293

In [37]:
# are you intense or carefree
X_trainI, X_testI, y_trainI, y_testI = train_test_split(int_x, int_y, test_size=.3,random_state=0)
modI = linear_model.LogisticRegression()
fitI = modI.fit(X_trainI, y_trainI)
y_predI = fitI.predict_proba(X_testI)
roc_auc_score(y_testI,y_predI[:,1])

/Users/kylemix/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.62120568667423814

In [38]:
# is art important to you?
X_trainA, X_testA, y_trainA, y_testA = train_test_split(art_x, art_y, test_size=.3,random_state=0)
modA = linear_model.LogisticRegression()
fitA = modA.fit(X_trainA, y_trainA)
y_predA = fitA.predict_proba(X_testA)
roc_auc_score(y_testA,y_predA[:,1])

/Users/kylemix/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.72158578662115813

In [52]:
# which would you rather have flight or invisibility?
X_trainF, X_testF, y_trainF, y_testF = train_test_split(flight_x, flight_y, test_size=.3,random_state=0)
modF = linear_model.LogisticRegression()
fitF = modF.fit(X_trainF, y_trainF)
y_predF = fitF.predict_proba(X_testF)
roc_auc_score(y_testF,y_predF[:,1])

/Users/kylemix/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.57431713014793095

In [40]:
# are you annoyed by super logical people?
X_trainL, X_testL, y_trainL, y_testL = train_test_split(logic_x, logic_y, test_size=.3,random_state=0)
modL = linear_model.LogisticRegression()
fitL = modL.fit(X_trainL, y_trainL)
y_predL = fitL.predict_proba(X_testL)
roc_auc_score(y_testL,y_predL[:,1])
# NOT INCLUDED

/Users/kylemix/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.62609822593727293

In [41]:
# which do you spend more money on, clothes or food?
X_trainC, X_testC, y_trainC, y_testC = train_test_split(clothes_x, clothes_y, test_size=.3,random_state=0)
modC = linear_model.LogisticRegression()
fitC = modC.fit(X_trainC, y_trainC)
y_predC = fitC.predict_proba(X_testC)
roc_auc_score(y_testC,y_predC[:,1])

/Users/kylemix/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6839064448525034

In [42]:
# would you date someone that kept a gun in the house?
X_trainG, X_testG, y_trainG, y_testG = train_test_split(guns_x, guns_y, test_size=.3,random_state=0)
modG = linear_model.LogisticRegression()
fitG = modG.fit(X_trainG, y_trainG)
y_predG = fitG.predict_proba(X_testG)
roc_auc_score(y_testG,y_predG[:,1])

/Users/kylemix/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6983180807258017

In [43]:
# would not enough affection or not enough space bother you more in a relationship?
X_trainS, X_testS, y_trainS, y_testS = train_test_split(space_x, space_y, test_size=.3,random_state=0)
modS = linear_model.LogisticRegression()
fitS = modS.fit(X_trainS, np.ravel(y_trainS))
y_predS = fitS.predict_proba(X_testS)
roc_auc_score(y_testS,y_predS[:,1])
# NOT INCLUDED

0.65691231188861421

In [44]:
# where you picked on a lot in school?
X_trainP, X_testP, y_trainP, y_testP = train_test_split(bullied_x, bullied_y, test_size=.3,random_state=0)
modP = linear_model.LogisticRegression()
fitP = modP.fit(X_trainP, y_trainP)
y_predP = fitP.predict_proba(X_testP)
roc_auc_score(y_testP,y_predP[:,1])

/Users/kylemix/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.65966312918105119

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlpP = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(5, 2), random_state=1)

In [ ]:
mlpfitp = mlpP.fit(X_trainP, np.ravel(y_trainP))

In [ ]:
y_predPmlp = fitP.predict_proba(X_testP)

In [ ]:
roc_auc_score(y_testP,y_predPmlp[:,1])

In [ ]:
X_trainDm, X_testDm, y_trainDm, y_testDm = train_test_split(dan_x, np.ravel(dan_y), test_size=.3,random_state=0)
modDm = MLPClassifier(solver='adam', alpha=0.0001,
                     hidden_layer_sizes=(500, 20), random_state=5, verbose=True)
fitDm = modDm.fit(X_trainDm, y_trainDm)
y_predDm = fitDm.predict(X_testDm)
roc_auc_score(y_testDm,y_predDm)

In [ ]:
# how do you feel about open relationships?  0.85394482894181467
# do you google someone before the first date? 0.61841297722880673
# male or female? 0.81857854533715779
# what is your stance on drug use?  0.8508118087357841
# orgasms are the most important part of sex  0.69458068866753209
# do you enjoy intense intellectual conversations? 0.64880138495641893
# is art important to you?  0.7350071409537583
# which would you rather have flight or invisibility? 0.5903666027237543
# are you annoyed by super logical people?  0.65179081131608485
# which do you spend more money on, clothes or food? 0.81218014182992082
# would you date someone that kept a gun in the house? 0.72021617752653999
# would not enough affection or not enough space bother you more in a relationship? 0.68686916947024468
# where you picked on a lot in school? 0.6789656815650138

In [ ]:
predictors = dfPred.columns

In [ ]:
dfPred.head()

In [ ]:
predictors

In [ ]:
q8155 = RadioField('On average, which best describes how often you GET WICKED DRUNK?', choices=[('1','Once every month or three.'),('2','Seldom or never.'),('3','Twice a week or more'),('4','Weekly / Bi-Weekly')])

In [ ]:
dfPred.head()

In [ ]:
fitR , fitD , fitK, fitB, fitA, fitG, fitP, fitS

In [ ]:
import pickle



In [ ]:
with open('models/fitR.pkl', 'wb') as picklefile1:
    pickle.dump(fitR, picklefile1)

In [ ]:
with open('models/fitD.pkl', 'wb') as picklefile2:
    pickle.dump(fitD, picklefile2)

In [ ]:
with open('models/fitK.pkl', 'wb') as picklefile3:
    pickle.dump(fitK, picklefile3)

In [ ]:
with open('models/fitB.pkl', 'wb') as picklefile4:
    pickle.dump(fitB, picklefile4)

In [ ]:
with open('models/fitA.pkl', 'wb') as picklefile5:
    pickle.dump(fitA, picklefile5)

In [ ]:
with open('models/fitG.pkl', 'wb') as picklefile6:
    pickle.dump(fitG, picklefile6)

In [ ]:
with open('models/fitP.pkl', 'wb') as picklefile7:
    pickle.dump(fitP, picklefile7)

In [ ]:
with open('models/fitS.pkl', 'wb') as picklefile8:
    pickle.dump(fitS, picklefile8)

In [ ]:
with open('models/fitL.pkl', 'wb') as picklefile9:
    pickle.dump(fitL, picklefile9)

In [ ]:
with open('models/fitC.pkl', 'wb') as picklefile10:
    pickle.dump(fitC, picklefile10)

In [ ]:

def plotRoc(test, score, title):
    fpr, tpr, thresholds = roc_curve(test, score)
    plt.figure(figsize=(10,10))
    plt.plot(fpr, tpr, color='red', label='ROC curve (area = %0.3f)' % roc_auc_score(test,score))
    plt.plot((0.0,1.0), color='steelblue', linestyle='--');
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc="lower right")
    plt.xlim([-0.02, 1.0])
    plt.ylim([0.0, 1.02])
    plt.title(title)
    plt.show();
    
def saveRocPlot(test, score, title,filename):
    fpr, tpr, thresholds = roc_curve(test, score)
    plt.figure(figsize=(10,10))
    plt.plot(fpr, tpr, color='red', label='ROC curve (area = %0.3f)' % roc_auc_score(test,score))
    plt.plot((0.0,1.0), color='steelblue', linestyle='--');
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc="lower right")
    plt.xlim([-0.02, 1.0])
    plt.ylim([0.0, 1.02])
    plt.title(title)
    plt.savefig(filename);

In [ ]:
plotRoc(y_testR,y_predR[:,1], 'How do you feel about open relationships?')
saveRocPlot(y_testR,y_predR[:,1], 'How do you feel about open relationships?','ronPlot.png')

In [ ]:
plotRoc(y_testD,y_predD[:,1],'Do you google before the first date?')
saveRocPlot(y_testD,y_predD[:,1],'Do you google before the first date?', 'danielPlot.png')

In [ ]:
plotRoc(y_testK,y_predK[:,1], 'Are you male or female?')
saveRocPlot(y_testK,y_predK[:,1], 'Are you male or female?', 'kylePlot.png')

In [ ]:
plotRoc(y_testB,y_predB[:,1], 'What is your feeling on drugs?')
saveRocPlot(y_testB,y_predB[:,1], 'What is your feeling on drugs?', 'bobPlot.png')

In [ ]:
plotRoc(y_testA,y_predA[:,1], 'Is art important to you?')
saveRocPlot(y_testA,y_predA[:,1], 'Is art important to you?', 'artPlot.png')

In [ ]:
import PIL


In [ ]:

from PIL import Image

basewidth = 3000
img = Image.open('rspcorr.png')
wpercent = (basewidth/float(img.size[0]))
hsize = int((float(img.size[1])*float(wpercent)))
img = img.resize((basewidth,hsize), PIL.Image.ANTIALIAS)
img.save('corrSmall.jpg')


In [48]:
dfLogic.head()

,q301_No,q301_Yes,q30169_Gross!,q30169_It's_okay.,q30169_Love_it!,q30169_Not_so_much.,q81_Lots,q81_Never,q81_Rarely,q81_Sometimes,q79_I_smoke_occasionally.,q79_I_smoke_regularly.,"q79_I_smoked_in_the_past,_but_no_longer.",q79_Never.,q77_Never,q77_Rarely,q77_Sometimes,q77_Very_often,q66506_Not_important.,q66506_Somewhat_important.,q66506_Very_important.,blank19,q60100_No,q60100_Yes,blank17,blank18,q43261_I_almost_always_do_what_my_parents_think_is_best.,q43261_I_always_do_what_my_parents_say.,q43261_I_am_my_own_person.,q43261_I_consider_their_opinion_but_go_my_own_way.,q4018_No,q4018_Yes,blank15,blank16,q393_No,q393_Yes,blank13,blank14,q37693_No.,"q37693_Yes,_I'd_be_playing_with_them.","q37693_Yes,_but_I_don't_like_video_games.","q37693_Yes,_but_I_would_not_play_that_much.",q358084_No,q358084_Yes,blank11,blank12,q35_Love,q35_Sex,blank9,blank10,q274_College,q274_Graduate_School,q274_High_School,q274_Junior_High,q19874_0-6_months,q19874_12+_months,q19874_6-12_months,q19874_I've_never_been_in_a_relationship,q1597_No,q1597_Yes,blank7,blank8,q15280_No,q15280_Yes,blank5,blank6,q136_No,q136_Yes,blank3,blank4,q123_No,q123_Yes,blank1,blank2,"q1128_No,_but_I_don't_think_it's_inherently_wrong.","q1128_No,_it's_wrong.","q1128_Yes,_but_only_if_everybody_knew.","q1128_Yes,_even_in_secret.",q1062_A_couple_times_a_week.,q1062_At_least_once_a_day.,q1062_Once_a_week_or_less.,q1062_Usually_daily.__I_skip_some.,q18698_G_+_V_=_1/T,q18698_Not_sure_/_wish_I_could_skip_this_one,q18698_PV_=_nRT,q18698_y=mx+b
5,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
10,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
12,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
14,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0


In [53]:
scores = [0.84875446544920385,0.60960339224287607,0.77391653191956677,0.84401671153177926,0.62120568667423814,
0.72158578662115813,0.6839064448525034,0.6983180807258017,0.65966312918105119]

In [54]:
scores.median()

AttributeError: 'list' object has no attribute 'median'

In [55]:
np.median(scores)

0.6983180807258017

In [56]:
np.mean(scores)

0.71788558102201983